In [1]:
import numpy as np
import pandas as pd
import math
import os

In [2]:
def getGetThreshold(depart,road,b,K,l,B_prime):#失败离开者编号、用户信息、用户报价、当前阶段、总阶段数、当前预算
    path=r'C:\Users\98306\Desktop\实验\Real map\poi次数分布(总次数为1241).xlsx'
    df=pd.read_excel(path)
    arr_df = df.to_numpy().flatten()
    total_value=0#总价值
    total_b=0#总报价
    U=2#效率上界
    L=1#效率下界
    st=np.zeros(road.shape[0])#标记数组
    for i in range(len(depart)):#最多把失败者全选了
        idx=-1
        cnt=0
        for j in depart:
            if st[j]==1:
                continue
            for k in range(road.shape[1]):
                if arr_df[k]==0:
                    road[j,k]=0
            if road[j].sum()/b[j]>cnt:
                idx=j
                cnt=road[j].sum()/b[j]
        if idx==-1 or b[idx]>2*(U/L)*road[idx].sum()*B_prime/(total_value+road[idx].sum()):
            break
        st[idx]=1
        total_value+=road[idx].sum()#加上idx边际价值
        total_b+=b[idx]#加上idx报价
        for j in range(road.shape[1]):#更新poi点次数
            if road[idx,j]>0:
                arr_df[k]-=1
    if total_b==0:
        return 0
    return (total_value/total_b)*((U/L)**(1-K))/2

In [7]:
#TDMC
file_name=np.array([700])
# file_name=np.array([600])
for i in file_name:
    root_path=os.path.join("C:\\Users\\98306\\Desktop\\实验\\final_test\\test",str(i))
    res,B_=0,0#平均价值、平均预算
    people=0
    for j in range(20):
        road_path=os.path.join(root_path,str(j)+'_road.xlsx')
        road=road=pd.read_excel(road_path)
        road=road.to_numpy()
        
        b_path=os.path.join(root_path,str(j)+'_b.xlsx')
        b=pd.read_excel(b_path,header=None)
        b=b.to_numpy().flatten()
        
        path=r'C:\Users\98306\Desktop\实验\Real map\poi次数分布(总次数为1241).xlsx'
        df=pd.read_excel(path)
        arr_df = df.to_numpy().flatten()
        B=6000
        epsilon=0.1#初始阈值
        Rho=epsilon#阈值
        l=int(math.log2(road.shape[0]+1))#计算阶段数
        K=l
        B_prime=B/(2**l)
        BB=B_prime#阶段总预算
        depart=[]#未被选中用户
        t=1#分位
        for k in range(road.shape[0]):
            for w in range(road.shape[1]):
                if arr_df[w]==0:
                    road[k,w]=0
            if road[k].sum()/b[k]>=Rho and B_prime>=road[k].sum()/Rho:
                res+=road[k].sum()
                B_prime-=road[k].sum()/Rho
                B-=road[k].sum()/Rho
                people+=1
                for w in range(road.shape[1]):
                    if road[k,w]>0:
                        arr_df[w]-=1
            else:
                depart.append(k)
            if k+1==t and 2*t<=road.shape[0]+1:
                Rho_prime=getGetThreshold(depart.copy(),road.copy(),b.copy(),K,l,BB)
                if Rho_prime!=0:
                    Rho=Rho_prime
                K-=1#更新当前阶段
                B_prime+=BB#预算翻倍
                BB*=2
                t*=2
        B_+=B
    print("用户数量:",i,"   平均价值：",res/20,"    平均支付预算:",6000-B_/20,"    平均选中人数:",people/20)

用户数量: 700    平均价值： 13358.164071971594     平均支付预算: 2615.213984654017     平均选中人数: 24.55


In [42]:
file_name=np.array([5,10,20,50,100,200,300,400,500])
U=0
L=1000000000
for i in file_name:
    root_path=os.path.join("C:\\Users\\98306\\Desktop\\实验\\final_test\\test",str(i))
    for j in range(20):
        road_path=os.path.join(root_path,str(j)+'_road.xlsx')
        road=pd.read_excel(road_path)
        road=road.to_numpy()
        
        b_path=os.path.join(root_path,str(j)+'_b.xlsx')
        b=pd.read_excel(b_path,header=None)
        b=b.to_numpy().flatten()
        for k in range(road.shape[0]):
            if road[k].sum()/b[k]>U:
                U=road[k].sum()/b[k]
            if road[k].sum()/b[k]<L:
                L=road[k].sum()/b[k]
    print("U:",U,"L:",L)            

U: 538.6625880971678 L: 9.255411371234318
U: 2449.727147004589 L: 8.55514405462632
U: 4728.5160645169635 L: 8.55514405462632
U: 6903.02795024604 L: 5.492324580899249
U: 6903.02795024604 L: 5.365957829055109
U: 13577.354563070568 L: 5.365957829055109
U: 13577.354563070568 L: 5.365957829055109
U: 18566.61807633209 L: 5.365957829055109
U: 18566.61807633209 L: 5.365957829055109


0.125